<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/lstm/exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用lstm生成文本。此示例使用的是many-to-one模型，输入多个词，输出一个词。

导入需要的包

In [18]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# 1 加载数据

In [19]:
data = """ Jack and Jill went up the hill\n
To fetch a pail of water\n
Jack fell down and broke his crown\n
And Jill came tumbling after\n """

## 使用keras的Tokenizer将文本转换为整数序列。

Tokenizer的原理是：按照词频对训练文本中的词降序排序，然后使用序号对词进行编码。例如：排名第一的词对应的整数值是1，排名第二的词对应的整数值是2。

fit_on_texts后有两个有用的输出：  
word_counts：词频统计结果。例如：('jack', 2), ('and', 3), ('jill', 2), ('went', 1), ('up', 1)。  
word_index：词和index的对应关系，也就是词和整数的对应的关系。先按照词频对训练文本中的词降序排序，然后按照顺序给词分配索引（从1开始，0不会分配任何词）。例如：'and': 1, 'jack': 2, 'jill': 3, 'went': 4, 'up': 5。  


In [26]:
# prepare the tokenizer on the source text
# 创建分词器 Tokenizer 对象，添加结束符
tokenizer = Tokenizer(oov_token='<END>')

# 使用 data 训练分词器
tokenizer.fit_on_texts([data])

print(tokenizer.word_counts)
print(tokenizer.word_docs)
print(tokenizer.word_index) 
print(tokenizer.document_count)

OrderedDict([('jack', 2), ('and', 3), ('jill', 2), ('went', 1), ('up', 1), ('the', 1), ('hill', 1), ('to', 1), ('fetch', 1), ('a', 1), ('pail', 1), ('of', 1), ('water', 1), ('fell', 1), ('down', 1), ('broke', 1), ('his', 1), ('crown', 1), ('came', 1), ('tumbling', 1), ('after', 1)])
defaultdict(<class 'int'>, {'pail': 1, 'broke': 1, 'fell': 1, 'his': 1, 'crown': 1, 'a': 1, 'after': 1, 'came': 1, 'fetch': 1, 'to': 1, 'of': 1, 'tumbling': 1, 'and': 1, 'jill': 1, 'went': 1, 'down': 1, 'hill': 1, 'jack': 1, 'water': 1, 'up': 1, 'the': 1})
{'<END>': 1, 'and': 2, 'jack': 3, 'jill': 4, 'went': 5, 'up': 6, 'the': 7, 'hill': 8, 'to': 9, 'fetch': 10, 'a': 11, 'pail': 12, 'of': 13, 'water': 14, 'fell': 15, 'down': 16, 'broke': 17, 'his': 18, 'crown': 19, 'came': 20, 'tumbling': 21, 'after': 22}
1


## 确定词汇量

作用：
1. 后续定义模型时，词嵌入层需要用到。
2. 使用one-hot编码编码输出的词。

In [27]:
# 确定词汇量（原始文本中词去重后的数量）
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 23


## 基于行创建（整数）序列




按照行创建序列，将原序列[2, 1, 3, 4, 5, 6, 7]转换成如下形式：  
[2],   
[2, 1],   
[2, 1, 3],   
[2, 1, 3, 4],   
[2, 1, 3, 4, 5],   
[2, 1, 3, 4, 5, 6],   
[2, 1, 3, 4, 5, 6, 7]

In [31]:
sequences = list()
for line in data.split('\n'):
  if len(line.strip()) > 0:
    line += ' <END>'
    # 将文本转换为（整数）序列
    encoded = tokenizer.texts_to_sequences([line])[0]  # tokenizer.texts_to_sequences([line])的结果[[2, 1, 3, 4, 5, 6, 7]]
    print(encoded)
    for i in range(1, len(encoded)+1):
      sequence = encoded[:i]
      sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
print(sequences[:10])

[3, 2, 4, 5, 6, 7, 8, 1]
[9, 10, 11, 12, 13, 14, 1]
[3, 15, 16, 2, 17, 18, 19, 1]
[2, 4, 20, 21, 22, 1]
Total Sequences: 29
[[3], [3, 2], [3, 2, 4], [3, 2, 4, 5], [3, 2, 4, 5, 6], [3, 2, 4, 5, 6, 7], [3, 2, 4, 5, 6, 7, 8], [3, 2, 4, 5, 6, 7, 8, 1], [9], [9, 10]]


## 填充序列

由于新序列的长度不一致，因此需要对序列进行填充使得所有序列长度相同。

使用keras的pad_sequences来进行填充。

填充结果：  
[ 0  0  0  0  0  0  2]  
 [ 0  0  0  0  0  2  1]  
 [ 0  0  0  0  2  1  3]  
 [ 0  0  0  2  1  3  4]  
 [ 0  0  2  1  3  4  5]  
 [ 0  2  1  3  4  5  6]  
 [ 2  1  3  4  5  6  7]  


In [32]:
# pad input sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre') # 左边填充0
print('Max Sequence Length: %d' % max_length)
print(sequences[:10])

Max Sequence Length: 8
[[ 0  0  0  0  0  0  0  3]
 [ 0  0  0  0  0  0  3  2]
 [ 0  0  0  0  0  3  2  4]
 [ 0  0  0  0  3  2  4  5]
 [ 0  0  0  3  2  4  5  6]
 [ 0  0  3  2  4  5  6  7]
 [ 0  3  2  4  5  6  7  8]
 [ 3  2  4  5  6  7  8  1]
 [ 0  0  0  0  0  0  0  9]
 [ 0  0  0  0  0  0  9 10]]


# 创建输入输出

从序列中数据分割出输入和输出数据。


```
    X       y  
[0 0 0 0 0 0]  2  
[0 0 0 0 0 2]  1  
[0 0 0 0 2 1]  3  
[0 0 0 2 1 3]  4  
[0 0 2 1 3 4]  5 
```


 


In [33]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
print(X[:5])
print(y[:5])
y = to_categorical(y, num_classes=vocab_size) # 对输出进行one-hot编码

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 3]
 [0 0 0 0 0 3 2]
 [0 0 0 0 3 2 4]
 [0 0 0 3 2 4 5]]
[3 2 4 5 6]


# 定义模型

Embedding：  
输入shape：形如（samples，sequence_length）的2D张量  
输出shape：形如(samples, sequence_length, output_dim)的3D张量  
嵌入层将正整数（下标）转换为具有固定大小的向量，如[[4],[20]]->[[0.25,0.1],[0.6,-0.2]]。[Embedding详解](https://blog.csdn.net/jiangpeng59/article/details/77533309) 



LSTM：  
输入shape：(samples, time_steps, input_dim)  
输出shape：(samples, output_dim)

*samples表示样本数量*


In [34]:
def define_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_length-1))
  model.add(LSTM(50))
  model.add(Dense(vocab_size, activation='softmax'))
  # compile network
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # summarize defined model 输出模型各层的参数状况
  model.summary()
  # plot_model(model, to_file='model.png', show_shapes=True)
  return model

In [35]:
# define model
model = define_model(vocab_size, max_length)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 10)             230       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 23)                1173      
Total params: 13,603
Trainable params: 13,603
Non-trainable params: 0
_________________________________________________________________


# 训练模型

In [36]:
# fit network 训练模型
model.fit(X, y, epochs=500, verbose=0)
# summarize performance of the model 总结模型性能
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 89.66%


# 使用模型进行预测

In [38]:
# generate a sequence from a language model
# 给定一个词，生成其后的词，形成句子
def generate_seq(model, tokenizer, max_length, seed_text, max_words):
  in_text = seed_text
  out_word = ''
  num_words = 0
  # generate a fixed number of words
  while True:
    if out_word == '<END>' or num_words >= max_words:
      break
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # pre-pad sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    # predict probabilities for each word
    # yhat = model.predict_classes(encoded, verbose=0)
    yhat = np.argmax(model.predict(encoded))
    # map predicted word index to word
    for word, index in tokenizer.word_index.items():
      if index == yhat:
        out_word = word
        break
    # append to input
    in_text += ' ' + out_word
    num_words += 1
  return in_text

In [40]:
# evaluate model
# 输入空，预测其后的词，形成句子，句子最大长度为10
print(generate_seq(model, tokenizer, max_length-1, '', 10)) 
# 输入Jill，预测其后的词，形成句子，句子最大长度为4
print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4))

 jack and jill went up the hill <END>
Jill jack came tumbling after
